In [ ]:
#!pip uninstall -y gmusicapi
#!pip uninstall -y google-cloud
#!pip uninstall -y google

#!pip install google-cloud
#!pip install google

#!pip install gmusicapi

#!pip install pandas

In [ ]:
from gmusicapi import Mobileclient

In [ ]:
import getpass

In [ ]:
username = getpass.getpass()

In [ ]:
pw = getpass.getpass()

In [ ]:
from gmusicapi import Mobileclient

api = Mobileclient()
logged_in = api.login(username, pw, Mobileclient.FROM_MAC_ADDRESS)

In [ ]:
logged_in

In [ ]:
from pprint import pprint
from toolz import pluck
from pandas import DataFrame
import pandas as pd
pd.set_option('display.max_rows', 5000)

In [ ]:
#cols = ['artist', 'year', 'album', 'trackNumber', 'title', 'playCount']
#df = DataFrame(list(pluck(cols, library, None)), columns=cols)
#df['playCount'] = df['playCount'].fillna(0).astype(int)
#df

In [ ]:
#df[df['playCount'] > 0].sort_values('playCount', ascending=False).reset_index(drop=True)

In [ ]:
#api.get_all_playlists()

In [ ]:
#api.get_all_stations()

In [ ]:
from IPython.display import display, Image, HTML
from jinja2 import Template

In [ ]:
#!pip install git+git://github.com/ojii/pymaging.git#egg=pymaging
#!pip install git+git://github.com/ojii/pymaging-png.git#egg=pymaging-png

In [ ]:
import qrcode
import qrcode.image.svg
from qrcode.image.pure import PymagingImage

In [ ]:
lib = api.get_all_songs()
thumbs_up = [t for t in lib if t.get('rating') == '5']

In [ ]:
thumbs_up

In [ ]:
cols = ['albumArtRef', 'albumId', 'albumArtist', 'year', 'album', 'id', 'trackNumber', 'artist', 'title', 'playCount']
df = DataFrame(list(pluck(cols, thumbs_up, None)), columns=cols)
df['playCount'] = df['playCount'].fillna(0).astype(int)
df['albumArt'] = df['albumArtRef'].apply(lambda v: v[0]['url'] if v else None)
df.drop('albumArtRef', axis=1, inplace=True)

In [ ]:
df

In [ ]:
df['qrcode'] = df['id'].apply(lambda _id: qrcode.make('qqqr:gmusic:track:' + _id, border=0, image_factory=PymagingImage))

In [ ]:
row = df.iloc[0]

In [ ]:
img = row.qrcode.get_image()

In [ ]:
import cStringIO
import base64

In [ ]:
def img_to_b64(img):
    buf = cStringIO.StringIO()
    img.save(buf, format='png')
    img_b64 = base64.b64encode(buf.getvalue())
    return img_b64

In [ ]:
#_ = Template("""
#<div style="width: 100px">
#{% for row in rows %}
#    <div style="display: inline-block; width: 240px; height: 500px; padding: 8px; border: 1px dashed black">
#    
#        <div><img width="220" height="220" src="{{row.albumArt}}"></div>
#        <div style="height: 40px; font-family: Helvetica; font-size: 12px">
#            {{row.albumArtist}} &ndash; {{row.title}}
#        </div>
#        <div><img width="220" height="220" src="data:image/png;base64,{{img_to_b64(row.qrcode.get_image())}}"></div>
#    
#    </div>
#{% endfor %}
#</div>
#""")

In [ ]:
template = Template("""
<div style="width: 1030px">
{% for row in rows %}
    <div style="display: inline-block; width: 320px; height: 640px; padding: 8px; border: 1px dashed black">
    
        <div><img width="300" height="300" src="{{row.albumArt}}"></div>
        <div style="height: 40px; font-family: Helvetica; font-size: 12px">
            {{row.albumArtist}} &ndash; {{row.title}}
        </div>
        <div><img width="300" height="300" src="data:image/png;base64,{{img_to_b64(row.qrcode.get_image())}}"></div>
    
    </div>
{% endfor %}
</div>
""")

In [ ]:
rows = [row for _, row in df.iterrows()]
display(HTML(template.render(rows=rows, img_to_b64=img_to_b64)))

In [ ]:
html = template.render(rows=rows, img_to_b64=img_to_b64)

In [1]:
#with open('qralbums.html', 'w') as f:
#    f.write(html)